In [150]:
import chess.pgn
import csv
import pandas as pd
import os
import io

In [137]:
pgn = open("russian_chess.pgn")

In [139]:
games = []
for game in pgn:
    curr = chess.pgn.read_game(pgn)
    games.append(curr)

In [112]:
len(games)

23

In [114]:
first_game = games[0]

In [ ]:
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
    print(move)
    print(board.fen())

In [154]:
def extract_combined_moves_and_comments(pgn_text):
    # Convert the string to a StringIO object
    pgn = io.StringIO(pgn_text)
    
    # Read the game
    game = chess.pgn.read_game(pgn)
    
    combined_moves_comments = []
    last_move = ""
    last_comment = ""
    move_number = 1
    
    node = game
    while not node.is_end():
        next_node = node.variation(0)
        move = node.board().san(next_node.move)
        comment = next_node.comment
        
        if node.board().turn == chess.BLACK:
            # Handle White's move
            if last_move:
                player = "White" if move_number % 2 == 1 else "Black"
                combined_move = f"{last_move} {move}"
                combined_comment = comment if comment else last_comment
                combined_moves_comments.append((move_number,player, combined_move, combined_comment))
                last_move = ""
                last_comment = ""
                move_number += 1
            else:
                last_move = move
                last_comment = comment
        else:
            # Handle Black's move
            if last_move:
                player = "White" if move_number % 2 == 1 else "Black"
                combined_move = f" {last_move} {move}"
                combined_comment = comment if comment else last_comment
                combined_moves_comments.append((move_number,player, combined_move, combined_comment))
                last_move = ""
                last_comment = ""
                move_number += 1
            else:
                last_move = move
                last_comment = comment
        
        node = next_node
    
    # Handle case where the game ends after White's move
    if last_move:
        player = "White" if move_number % 2 == 1 else "Black"
        combined_move = f" {last_move}"
        combined_comment = last_comment
        combined_moves_comments.append((move_number, player,combined_move, combined_comment))
    
    return combined_moves_comments

pgn_text = """Your PGN text goes here"""
combined_moves_comments = extract_combined_moves_and_comments(pgn_text)

# Print combined moves and comments
for entry in combined_moves_comments:
    print(entry)


In [147]:
def read_pgn_and_write_to_csv(pgn_file_path, csv_file_path):
    with open(pgn_file_path, 'r', encoding='utf-8') as pgn:
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = csv.writer(csvfile)
            # Write the headers
            csvwriter.writerow(['Game Number', 'Move Number', 'Player', 'Move', 'Comment'])
            
            game_number = 1
            while True:
                game = chess.pgn.read_game(pgn)
                if game is None:
                    break  # End of file or no more games
                
                combined_moves_comments = extract_combined_moves_and_comments(str(game))
                
                # Write each move's data to the CSV, prepending the game number
                for move_number, player, move, comment in combined_moves_comments:
                    csvwriter.writerow([game_number, move_number, player, move, comment])
                
                game_number += 1

In [148]:
read_pgn_and_write_to_csv('/Users/mzk/Documents/566 Project/russian_chess.pgn', 'russian_chess.csv')

In [152]:
def process_pgn_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.pgn'):
            pgn_file_path = os.path.join(directory, filename)
            csv_file_path = os.path.join(directory, os.path.splitext(filename)[0] + '.csv')

            # Now read the PGN file and write to the corresponding CSV file
            read_pgn_and_write_to_csv(pgn_file_path, csv_file_path)
            print(f"Processed {filename} to {os.path.splitext(filename)[0] + '.csv'}")

In [155]:
directory = '/Users/mzk/Documents/566 Project'
process_pgn_files(directory)

Processed linares_2002.pgn to linares_2002.csv
Processed middleg.pgn to middleg.csv


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 7062: invalid continuation byte

In [157]:
#read as df limit 1000
df = pd.read_csv('linares_2002.csv', nrows=1000)